In [1]:
from transformers import BasicTokenizer, Trainer
from datasets import concatenate_datasets, load_from_disk
import kagglehub
import optuna
import torch
import math
import base

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [2]:
base.reset_seed()

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


In [4]:
my_glove = kagglehub.dataset_download("thanakomsn/glove6b300dtxt")
print(my_glove)

/home/jovyan/.cache/kagglehub/datasets/thanakomsn/glove6b300dtxt/versions/1


In [5]:
GLOVE_FILE = f"{my_glove}/glove.6B.300d.txt"
DATASET = "trec"

In [6]:
train_data = load_from_disk(f"~/data/{DATASET}/train-logits_coarse")
eval_data = load_from_disk(f"~/data/{DATASET}/eval-logits_coarse")
test_data = load_from_disk(f"~/data/{DATASET}/test-logits_coarse")

all_train_data = load_from_disk(f"~/data/{DATASET}/train-logits-augmented_coarse")

all_data = concatenate_datasets([load_from_disk(file) for file in [f"~/data/{DATASET}/eval-logits_coarse", f"~/data/{DATASET}/test-logits_coarse", f"~/data/{DATASET}/train-logits-augmented_coarse"]])
tokenizer = BasicTokenizer(do_lower_case=True)

In [7]:
train_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), train_data))
eval_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), eval_data))
test_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), test_data))

all_train_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), all_train_data))

all_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), all_data))

In [8]:
vocab = base.get_vocab(all_data_tokens)

In [9]:
word_index = dict(zip(vocab, range(len(vocab))))

In [10]:
embeddings_index = base.get_embeddings_indeces(GLOVE_FILE)

Found 400000 word vectors.


In [11]:
print(len(vocab))
num_tokens = len(vocab) + 2
embedding_dim = 300

8766


In [12]:
embedding_matrix = base.get_embedding_matrix(num_tokens, embedding_dim, word_index, embeddings_index)

Converted 8551 words (215) misses


In [13]:
train_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),train_data_tokens))
eval_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),eval_data_tokens))
test_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),test_data_tokens))

all_train_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),all_train_data_tokens))

In [14]:
train_padded_data = list(map(lambda x: base.padd(x,60), train_data_index))
eval_padded_data = list(map(lambda x: base.padd(x,60), eval_data_index))
test_padded_data = list(map(lambda x: base.padd(x,60), test_data_index))

all_train_padded_data = list(map(lambda x: base.padd(x,60), all_train_data_index))

In [15]:
train_data = train_data.add_column("input_ids", train_padded_data)
eval_data = eval_data.add_column("input_ids", eval_padded_data)
test_data = test_data.add_column("input_ids", test_padded_data)

all_train_data = all_train_data.add_column("input_ids", all_train_padded_data)

In [16]:
num_epochs = 15
batch_size = 128

In [17]:
#Nápočet epoch na steps
data_length = len(train_data)
min_r = math.ceil(data_length/batch_size)*5
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

In [18]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up)
    }   
    print(f"Trial {trial.number} with params: {params}")
    return params

In [19]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [20]:
def get_BiLSTM():
    return base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=6, freeze_embed=True)

In [21]:
base.reset_seed()

In [22]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base_coarse_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-base_coarse_hp-search", epochs=num_epochs, batch_size=batch_size)

In [23]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM()
)
  

In [24]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Base",
    n_trials=150
)

[I 2025-03-21 23:19:55,926] A new study created in memory with name: Base


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 3}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.586400,1.352458,0.472044,0.556011,0.364247,0.328637
2,1.156400,0.997047,0.615949,0.554059,0.514145,0.522096
3,0.851100,0.799823,0.710357,0.610599,0.604893,0.601129
4,0.663900,0.705013,0.754354,0.655110,0.639060,0.640721
5,0.583900,0.636923,0.770852,0.654833,0.658059,0.654533
6,0.534500,0.611490,0.776352,0.654750,0.663254,0.657938
7,0.474200,0.603255,0.776352,0.657674,0.663115,0.658393
8,0.437400,0.586060,0.788268,0.665197,0.675951,0.667878
9,0.407900,0.596886,0.791017,0.662209,0.678182,0.667026
10,0.375600,0.595865,0.787351,0.662146,0.671639,0.665349


[I 2025-03-21 23:21:09,272] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.0007875660249889869, 'weight_decay': 0.001, 'warmup_steps': 0}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.038800,0.626637,0.777269,0.677974,0.662374,0.662969
2,0.494200,0.515944,0.829514,0.831084,0.780970,0.796825
3,0.300400,0.486505,0.854262,0.826871,0.821324,0.820818
4,0.185100,0.476787,0.863428,0.874071,0.832720,0.849403
5,0.106600,0.503219,0.872594,0.861871,0.833131,0.845198
6,0.066700,0.574702,0.859762,0.834455,0.826322,0.828312
7,0.041800,0.680997,0.863428,0.860688,0.825170,0.839177
8,0.020400,0.763894,0.866178,0.859573,0.827334,0.840852
9,0.005100,0.755411,0.880843,0.878781,0.839831,0.855599
10,0.002200,0.791907,0.873511,0.875206,0.833517,0.850454


[I 2025-03-22 02:07:17,104] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.004203220195458997, 'weight_decay': 0.002, 'warmup_steps': 3}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.056300,0.788670,0.735105,0.665197,0.624790,0.629811
2,0.549400,0.549000,0.807516,0.790293,0.727738,0.740089
3,0.333600,0.477101,0.837764,0.812529,0.788987,0.796627
4,0.220400,0.487036,0.864345,0.879198,0.806110,0.830936
5,0.129800,0.522089,0.866178,0.871493,0.827085,0.844657
6,0.085100,0.558470,0.865261,0.853746,0.818877,0.832638
7,0.036400,0.647904,0.869844,0.880920,0.811967,0.835323
8,0.012000,0.679467,0.872594,0.872758,0.822617,0.842209
9,0.004100,0.723174,0.878093,0.876774,0.828272,0.847002
10,0.007000,0.723778,0.868011,0.867278,0.820652,0.838661


[I 2025-03-22 02:08:06,694] Trial 149 pruned. 


In [25]:
print(best_trial)

BestRun(run_id='123', objective=0.8691821658235614, hyperparameters={'learning_rate': 0.004483170022431039, 'weight_decay': 0.0, 'warmup_steps': 3}, run_summary=None)


In [26]:
base.reset_seed()

In [27]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill_coarse_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-distill_coarse_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

In [28]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

In [29]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [30]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM(),
)
  

In [31]:
best_trial2 = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill",
    n_trials=150
)

[I 2025-03-22 02:08:07,009] A new study created in memory with name: Distill


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 3, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.698800,3.231866,0.442713,0.386394,0.334541,0.279822
2,2.846600,2.454531,0.582951,0.543396,0.477618,0.478625
3,2.214700,1.975077,0.684693,0.574947,0.587045,0.580133
4,1.735800,1.709124,0.739688,0.642520,0.628358,0.630368
5,1.504400,1.546215,0.763520,0.645572,0.653665,0.647060


[I 2025-03-22 02:08:29,737] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.00010255552094216992, 'weight_decay': 0.0, 'warmup_steps': 4, 'lambda_param': 0.6000000000000001, 'temperature': 5.5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.487800,1.533138,0.753437,0.652568,0.641433,0.641152
2,1.166600,1.157194,0.804766,0.679581,0.691078,0.678857
3,0.712700,0.927107,0.852429,0.718994,0.727480,0.721614
4,0.457800,0.805847,0.864345,0.870836,0.779827,0.801898
5,0.266000,0.701516,0.881760,0.892150,0.831941,0.853823
6,0.159300,0.692707,0.889093,0.898597,0.836612,0.859288
7,0.109700,0.687124,0.880843,0.893479,0.830185,0.853274
8,0.087500,0.679413,0.885426,0.893620,0.843168,0.862388
9,0.074500,0.653866,0.882676,0.891903,0.831411,0.853397
10,0.068600,0.650024,0.891842,0.900005,0.838940,0.861125


[I 2025-03-22 05:10:29,088] Trial 148 finished with value: 0.8685321407320162 and parameters: {'learning_rate': 0.004724787958470258, 'weight_decay': 0.002, 'warmup_steps': 3, 'lambda_param': 0.8, 'temperature': 3.5}. Best is trial 53 with value: 0.8696635593759803.


Trial 149 with params: {'learning_rate': 0.0016430785628756667, 'weight_decay': 0.0, 'warmup_steps': 2, 'lambda_param': 0.30000000000000004, 'temperature': 4.0}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.854700,1.871830,0.696609,0.597755,0.597526,0.589403
2,1.450000,1.534917,0.748854,0.626956,0.648099,0.629862
3,1.050600,1.123489,0.820348,0.690292,0.701492,0.692906
4,0.797400,1.018842,0.839597,0.712681,0.713959,0.710886
5,0.635400,0.963742,0.844180,0.717051,0.718968,0.715714
6,0.487500,0.893295,0.849679,0.883357,0.751384,0.768143
7,0.336000,0.856675,0.861595,0.862307,0.780704,0.799613
8,0.244800,0.866357,0.861595,0.877316,0.816917,0.836790
9,0.191600,0.806715,0.863428,0.878302,0.825112,0.845453
10,0.139100,0.841721,0.857012,0.859997,0.821717,0.836636


[I 2025-03-22 05:11:37,282] Trial 149 pruned. 


In [32]:
print(best_trial2)

BestRun(run_id='53', objective=0.8696635593759803, hyperparameters={'learning_rate': 0.004269637593700472, 'weight_decay': 0.004, 'warmup_steps': 3, 'lambda_param': 0.1, 'temperature': 3.0}, run_summary=None)


In [33]:
#Nápočet epoch na steps
data_length = len(all_train_data)
min_r = math.ceil(data_length/batch_size)*5
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

In [34]:
base.reset_seed()

In [35]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base_coarse_aug_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-base_coarse_aug_hp-search", epochs=num_epochs, batch_size=batch_size)

In [36]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

In [37]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [38]:
trainer = Trainer(
    args=training_args,
    train_dataset=all_train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM(),
)
  

In [39]:
best_trial3 = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Base-aug",
    n_trials=150
)

[I 2025-03-22 05:11:37,662] A new study created in memory with name: Base-aug


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 23}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.825900,0.584778,0.794684,0.668485,0.679396,0.673175
2,0.392200,0.513855,0.836847,0.872888,0.722258,0.727334
3,0.275500,0.543900,0.839597,0.843093,0.774421,0.790163
4,0.194900,0.565894,0.842346,0.861462,0.808615,0.828235
5,0.141700,0.583311,0.852429,0.868792,0.826644,0.843027


[I 2025-03-22 05:12:51,535] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.0007875660249889869, 'weight_decay': 0.001, 'warmup_steps': 4}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.455800,0.468828,0.876260,0.882508,0.810374,0.831575
2,0.105300,0.525199,0.870761,0.866727,0.836070,0.846454
3,0.048900,0.640151,0.878093,0.857704,0.850068,0.851577
4,0.023800,0.650993,0.878093,0.875697,0.837610,0.852683
5,0.015400,0.791981,0.870761,0.881685,0.833690,0.850197
6,0.009500,0.813271,0.890009,0.899437,0.847083,0.867033
7,0.007200,0.888969,0.880843,0.877400,0.840018,0.854662
8,0.002800,0.965374,0.889093,0.884895,0.846375,0.861564
9,0.003000,1.025497,0.884510,0.894583,0.843240,0.862242
10,0.002300,0.905758,0.894592,0.890246,0.850861,0.866767


[I 2025-03-22 11:09:02,990] Trial 148 finished with value: 0.8660006581303742 and parameters: {'learning_rate': 0.0025644695728777034, 'weight_decay': 0.006, 'warmup_steps': 6}. Best is trial 61 with value: 0.8887268625210981.


Trial 149 with params: {'learning_rate': 0.0021384459068938236, 'weight_decay': 0.005, 'warmup_steps': 4}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.479300,0.496594,0.861595,0.866208,0.790251,0.809739
2,0.123700,0.480494,0.875344,0.876147,0.846684,0.858662
3,0.054100,0.559513,0.881760,0.866647,0.861056,0.862620
4,0.024500,0.688874,0.879010,0.891614,0.849169,0.864993
5,0.017000,0.824152,0.873511,0.875067,0.836693,0.849233


[I 2025-03-22 11:10:06,390] Trial 149 pruned. 


In [40]:
print(best_trial3)

BestRun(run_id='61', objective=0.8887268625210981, hyperparameters={'learning_rate': 0.004589457243558559, 'weight_decay': 0.01, 'warmup_steps': 22}, run_summary=None)


In [41]:
base.reset_seed()

In [42]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill_coarse_aug_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-distill_coarse_aug_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

In [43]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

In [44]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [45]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=all_train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM(),
)
  

In [46]:
best_trial4 = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill-aug",
    n_trials=150
)

[I 2025-03-22 11:10:06,668] A new study created in memory with name: Distill-aug


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 23, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.859400,1.294197,0.783685,0.657775,0.671114,0.664079
2,0.848500,1.055298,0.829514,0.702343,0.708354,0.704396
3,0.630800,0.929389,0.851512,0.715117,0.726351,0.719966
4,0.482200,0.879125,0.849679,0.852295,0.771285,0.789640
5,0.368300,0.837346,0.860678,0.872959,0.824425,0.842884


[I 2025-03-22 11:11:14,448] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.00010255552094216992, 'weight_decay': 0.0, 'warmup_steps': 27, 'lambda_param': 0.6000000000000001, 'temperature': 5.5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.852200,0.688981,0.880843,0.887030,0.814404,0.836180
2,0.211500,0.580507,0.900092,0.896368,0.865038,0.878399
3,0.130300,0.515500,0.909258,0.902953,0.873157,0.885767
4,0.099600,0.540241,0.905591,0.909112,0.861644,0.879553
5,0.087300,0.525096,0.905591,0.912768,0.869743,0.887193
6,0.077400,0.524293,0.913841,0.916021,0.857785,0.878810
7,0.069600,0.513525,0.905591,0.908721,0.852382,0.872522
8,0.063900,0.509558,0.907424,0.912745,0.862559,0.881809
9,0.060200,0.512827,0.912007,0.916287,0.866052,0.885402
10,0.057400,0.507933,0.914757,0.917439,0.868418,0.887182


[I 2025-03-22 18:09:04,671] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.0017795655138595975, 'weight_decay': 0.002, 'warmup_steps': 15, 'lambda_param': 1.0, 'temperature': 4.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.054800,0.781391,0.870761,0.860386,0.806849,0.823513
2,0.283600,0.635593,0.887259,0.898147,0.855039,0.872450
3,0.157200,0.611040,0.891842,0.900613,0.859096,0.875734
4,0.115900,0.630078,0.895509,0.904981,0.861712,0.879331
5,0.095900,0.621274,0.890926,0.901536,0.857642,0.875585
6,0.083200,0.592813,0.895509,0.904742,0.861703,0.879275
7,0.074900,0.582884,0.898258,0.908077,0.863218,0.881658
8,0.068200,0.580514,0.897342,0.906400,0.863099,0.880779
9,0.064000,0.576732,0.898258,0.907164,0.863849,0.881546
10,0.060500,0.580563,0.899175,0.907542,0.864736,0.882207


[I 2025-03-22 18:11:10,000] Trial 149 pruned. 


In [47]:
print(best_trial4)

BestRun(run_id='44', objective=0.8969803270673352, hyperparameters={'learning_rate': 0.0037906172026621545, 'weight_decay': 0.0, 'warmup_steps': 16, 'lambda_param': 1.0, 'temperature': 4.0}, run_summary=None)


In [48]:
print("Best normal training score: ", best_trial)
print("Best distilation trianing score: ", best_trial2)
print("Best normal training score with augmentations: ", best_trial3)
print("Best distilation trianing score with augmentations: ",best_trial4)

Best normal training score:  BestRun(run_id='123', objective=0.8691821658235614, hyperparameters={'learning_rate': 0.004483170022431039, 'weight_decay': 0.0, 'warmup_steps': 3}, run_summary=None)
Best distilation trianing score:  BestRun(run_id='53', objective=0.8696635593759803, hyperparameters={'learning_rate': 0.004269637593700472, 'weight_decay': 0.004, 'warmup_steps': 3, 'lambda_param': 0.1, 'temperature': 3.0}, run_summary=None)
Best normal training score with augmentations:  BestRun(run_id='61', objective=0.8887268625210981, hyperparameters={'learning_rate': 0.004589457243558559, 'weight_decay': 0.01, 'warmup_steps': 22}, run_summary=None)
Best distilation trianing score with augmentations:  BestRun(run_id='44', objective=0.8969803270673352, hyperparameters={'learning_rate': 0.0037906172026621545, 'weight_decay': 0.0, 'warmup_steps': 16, 'lambda_param': 1.0, 'temperature': 4.0}, run_summary=None)
